### Import modules necessary for interaction with Twitter API

In [1]:
import tweepy
import cnfg
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from pymongo import MongoClient
from tweepy import Stream
from tweepy.streaming import StreamListener
import json
from textblob import TextBlob

Twitter API authentication - Consumer API Keys and Access Token.  These 4 keys are given when twitter gives you access to their API

In [2]:
auth = tweepy.OAuthHandler('xxxx',
                           'xxxx')
auth.set_access_token('xxxx',
                      'xxxx')

api=tweepy.API(auth)

Store tweets in a Mongo Database

In [3]:
#Intialize Mongo Database
client = MongoClient('localhost', 27017)

In [4]:
#View list of available databases
client.list_database_names()

['2fri_db',
 'acosta_db',
 'admin',
 'avenatti_db',
 'config',
 'fox_db',
 'fri_db',
 'last_fri_db',
 'late_db',
 'local',
 'more_sat',
 'new_db',
 'sat_db',
 'sat_late',
 'thurs_db',
 'thurs_test',
 'twitter_database',
 'wed_db']

Here is an example of how tweets were collected for each database

In [5]:
twitter_db = client['twitter_database']

In [6]:
#Initialize tweet collector
twitter_db_tweets = twitter_db.tweets

This is a test to collect a tweet from a username, and insert into one of our databases

In [ ]:
username = '@realDonaldTrump'
tweets = tweepy.Cursor(api.user_timeline, screen_name=username, tweet_mode='extended').items()
tweet_dict_list = []

for t in tqdm(tweets):
    d = {}
    d['created_at'] = t.created_at
    d['favorite_count'] = t.favorite_count
    d['retweet_count'] = t.retweet_count
    d['full_text'] = t.full_text
    d['screen_name'] = t.user.screen_name
    
    tweet_dict_list.append(d)

In [ ]:
#Insert first item from above into twitter_db database
twitter_db_tweets.insert_one(tweet_dict_list[0])

Now it's time to use our stream listener. This allows us to grab live tweets from any given key word search on twitter

## Stream Listener

In [8]:
class listener(StreamListener):

    def on_data(self, data):
        data = json.loads(data)
        
        # Pull the fields we want, and throw it into our mongodb database
        if data['lang'] == 'en':
            tweet_document = {}

            tweet_document['created_at'] = data['created_at']
            tweet_document['favorite_count'] = data['favorite_count']
            tweet_document['retweet_count'] = data['retweet_count']

            #Get full text if it is an extended tweet
            if 'extended_tweet' in data.keys():
                tweet_document['text'] = data['extended_tweet']['full_text']
            else:
                tweet_document['text'] = data['text']

            tweet_document['screen_name'] = data['user']['screen_name']

            text = TextBlob(tweet_document['text']).sentiment

            tweet_document['sentiment'] = text.polarity
            tweet_document['subjectivity'] = text.subjectivity

           # print(tweet_document)
           # print('\n')
            twitter_db_tweets.insert_one(tweet_document)
        
        return True

    def on_error(self, status):
        print(status)

In [9]:
#Start up the twitterStream
twitterStream = Stream(auth,listener())

We can specify by getting all tweets containing the following key words

In [ ]:
data = twitterStream.filter(track=['cnn','fox_news','wall street journal','@economist'])

In [ ]:
#Collect all the tweets, sorted by time created
tweet_collect= twitter_db['tweets'].find().sort([('created_at',-1)]).limit(10000)

In [ ]:
#Store the collection into a list
tweet_list = [item for item in tweet_collect]

In [ ]:
#Create a Pandas data frame from the list
tweet_df = pd.DataFrame(tweet_list)

Now we have a dataframe that has all the information needed to move forward with deeper analysis.